In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w4p2/sample.csv
/kaggle/input/2022-ai-w4p2/train.csv
/kaggle/input/2022-ai-w4p2/test.csv


In [2]:
import random
import torch
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('/kaggle/input/2022-ai-w4p2/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-w4p2/test.csv')
submit = pd.read_csv('/kaggle/input/2022-ai-w4p2/sample.csv')

In [4]:
train

,N,P,K,temperature,humidity,ph,rainfall,label
0,22,36,16,30.581395,50.771481,8.184229,64.585596,mothbeans
1,18,27,41,22.365094,92.308824,7.175344,104.821633,pomegranate
2,61,68,50,35.214628,91.497251,6.793245,243.074507,papaya
3,26,72,22,28.767949,37.577921,4.674942,91.720849,pigeonpeas
4,2,140,197,22.697801,92.822234,5.534567,105.050823,apple
...,...,...,...,...,...,...,...,...
1645,10,5,5,21.213070,91.353492,7.817846,112.983436,orange
1646,108,94,47,27.359116,84.546250,6.387431,90.812505,banana
1647,11,36,31,27.920633,51.779659,6.475449,100.258567,mango
1648,11,124,204,13.429886,80.066340,6.361141,71.400430,grapes


In [5]:
train.columns

Index(['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label'], dtype='object')

In [6]:
x_train = np.array(train[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']])
y_train = np.array(train[['label']])
x_test = np.array(test)

In [7]:
from sklearn.preprocessing import RobustScaler
trans1 = RobustScaler().fit(x_train)
x_train = trans1.transform(x_train)
trans2 = RobustScaler().fit(x_test)
x_test = trans2.transform(x_test)

In [8]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)

In [9]:
y_train

array([['mothbeans'],
       ['pomegranate'],
       ['papaya'],
       ...,
       ['mango'],
       ['grapes'],
       ['lentil']], dtype=object)

In [10]:
from sklearn.preprocessing import LabelEncoder
labels = LabelEncoder()
labels.fit(y_train)
label = labels.transform(y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
y_train = torch.LongTensor(label)

In [12]:
print(y_train)
print(y_train.unsqueeze(1))

tensor([13, 19, 17,  ..., 12,  7, 10])
tensor([[13],
        [19],
        [17],
        ...,
        [12],
        [ 7],
        [10]])


In [13]:
nb_class = 22
nb_data = len(y_train)
y_one_hot = torch.zeros(nb_data, nb_class)
y_one_hot.scatter_(1, y_train.unsqueeze(1),1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
import torch.nn.functional as F  # for softmax

W = torch.zeros((7, nb_class), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)
nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    
    # cost 표현번 1번 예시
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    # cost 표현법 2번 예시
    # cross_entropy를 사용하면 scatter 함수를 이용한 one_hot_encoding을 안해도 됨
    # cost = F.cross_entropy((x_train.matmul(W) + b), y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 3.091043
Epoch  100/10000 Cost: 2.020152
Epoch  200/10000 Cost: 1.585582
Epoch  300/10000 Cost: 1.324809
Epoch  400/10000 Cost: 1.149118
Epoch  500/10000 Cost: 1.022266
Epoch  600/10000 Cost: 0.926178
Epoch  700/10000 Cost: 0.850742
Epoch  800/10000 Cost: 0.789845
Epoch  900/10000 Cost: 0.739567
Epoch 1000/10000 Cost: 0.697278
Epoch 1100/10000 Cost: 0.661151
Epoch 1200/10000 Cost: 0.629875
Epoch 1300/10000 Cost: 0.602490
Epoch 1400/10000 Cost: 0.578276
Epoch 1500/10000 Cost: 0.556681
Epoch 1600/10000 Cost: 0.537276
Epoch 1700/10000 Cost: 0.519724
Epoch 1800/10000 Cost: 0.503754
Epoch 1900/10000 Cost: 0.489147
Epoch 2000/10000 Cost: 0.475724
Epoch 2100/10000 Cost: 0.463335
Epoch 2200/10000 Cost: 0.451857
Epoch 2300/10000 Cost: 0.441186
Epoch 2400/10000 Cost: 0.431232
Epoch 2500/10000 Cost: 0.421921
Epoch 2600/10000 Cost: 0.413187
Epoch 2700/10000 Cost: 0.404973
Epoch 2800/10000 Cost: 0.397232
Epoch 2900/10000 Cost: 0.389919
Epoch 3000/10000 Cost: 0.382998
Epoch 31

In [15]:
hypothesis = F.softmax(x_test.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)

In [16]:
predict = labels.inverse_transform(predict)

In [17]:
submit.columns

Index(['id', 'label'], dtype='object')

In [18]:
submit['label'] = predict
submission=submit.astype(np.str_)
submission.to_csv('submission.csv',mode='w',header=True,index=False)